# Exploring Property Data

Revisit the Property Assessment data. We are looking for some info on home prices - we want to narrow our search to a specific subset of homes.

Our hypothesis is that over time, prices in Edmonton will rise. If we can invest in a segment of the market that is undervalued, that segment will rise faster as it "catches up" with the rest. We don't know if this hypothesis is true in reality - in the future we could do some predictive modelling to try to test it. We are going with a HIPPO opinion - low valued segments will rise, so that is what we will look for in our data. 

We also are smaller investors. We don't have the up front cash to purchase big money properties and "swing for the fences", we would very much prefer to target properties that are relatively less expensive, to allow us to purchase more and diversify the risk. 

So, our goal is to identify which segments of real estate have values that are suitable for investment, based on our assumptions.  
(Note: There are differnt potential answers - I find a set of homes that may look good based on our criteria, there's probably others, it is not a yes/no answer)

<b>
Use: Attempt to break down and examine the data to find some property groups that appear to be solid investments, supported by the data you have. You can follow my example, or go your own way. There's probably many potential groups that make sense. The solution version has everything I did. Feel free to add more code blocks. I chopped them down so there are not a tonne of empty ones. As well, the exploration is somewhat open ended. If you want to closely follow along my steps, that's fine. You could do things differently, or in a somewhat different order, and that would be all good too. If you feel very comfortable doing things on your own, it is "better" practice to follow less closely. </b>

In [2]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np

#These 3 things allow for some prettier graphs, using a different visualization package - seaborn.
#This isn't required, we'll start using this a little bit as we go. Seaborn's big selling point
#is that it allows us to create many common graphs, that are very nice, relatively easily
#The third line just makes the default plot size larger, you can change the numbers if you want. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (20,5)

#### Fancy Graphing - Seaborn

Example of a seaborn distpolot - a histogram with a PDF overlayed on it. I use it in the solution.
In this example, the data is the first argument, the other two arguments are sets of "customizations" for the histogram and PDF (KDE) part.

You can use this, if you want, just change the data, and the labels and it should be usable for everything here.
There are more options and examples detailed here: https://seaborn.pydata.org/generated/seaborn.distplot.html

Generally, it doesn't really matter what tool you use to create visualizations. The thinkstats/thinkplot stuff fits in nicely with what is in the book and is pretty useful and easy to use. Later on, into the machine learning stuff we'll do all of our charting and plotting with Seaborn (or at least I will, you can use something else if you're adventerous), for now it is your choice unless specified. 

In [3]:
#sns.distplot(dtGlen["Assessed Value"], hist_kws={"label":"Condos"}, kde_kws={"label":"Condo PDF"},)
#sns.distplot(glen["Assessed Value"], hist_kws={"label":"Glenora Garage"}, kde_kws={"label":"Glenora Garage PDF"},)
#plt.show()

This function is a helper. 

One addition is the rnd=0 thing. That's how we can make one of those optional paramaters, if you don't provide anything there, the hist will have no rounding; if you provide a number, the histogram will round to that many digits. 

In [4]:
# Graphing Helper Function
def bigGraph(df_in, columnName, rnd=0):
    data = pd.Series(df_in[columnName])
    hist = thinkstats2.Hist(round(data, rnd))
    pmf = thinkstats2.Pmf(data)
    cdf = thinkstats2.Cdf(data)
    thinkplot.PrePlot(4, rows =2, cols=2)
    thinkplot.Hist(hist)
    thinkplot.SubPlot(2)
    thinkplot.Pmf(pmf)
    thinkplot.SubPlot(3)
    thinkplot.Cdf(cdf)
    thinkplot.SubPlot(4)
    pdf = thinkstats2.EstimatedPdf(data)
    thinkplot.Pdf(pdf)
    thinkplot.Config()
    return

In [5]:
#Load data - I used the odd varaible name because I want the final name to be "df" after I do any filtering, so it is easier. 
df_init = pd.read_csv("data/Assessments.csv")

## START HERE!

We can look at our data, and try to figure out a strategy to stack that paper. 

### Part 1: Load and Explore Data

Preview data and make sure the data present is what we want/need. 

In [6]:
#Examine the data a little bit to see what we have. 
df_init.head()


,Account Number,Suite,House Number,Street Name,Garage,Neighbourhood ID,Neighbourhood,Ward,Assessed Value,Latitude,Longitude
0,1066158,NaN,14904.0,167 AVENUE NW,N,NaN,NaN,NaN,86000,53.630497,-113.580474
1,10508233,NaN,16340.0,MARK MESSIER TRAIL NW,N,NaN,NaN,NaN,45000,53.617335,-113.605690
2,1194398,NaN,15617.0,83 STREET NW,N,2050.0,BELLE RIVE,Ward 3,285000,53.617007,-113.467976
3,1034214,NaN,10661.0,161 AVENUE NW,Y,3040.0,BEAUMARIS,Ward 3,635000,53.619978,-113.506990
4,1114701,NaN,15003.0,76 AVENUE NW,Y,4430.0,RIO TERRACE,Ward 5,407500,53.508644,-113.579897


In [7]:
# I like the .T view better
df_init.describe().T

,count,mean,std,min,25%,50%,75%,max
Account Number,384217.0,7.813424e+06,3.232763e+06,1.001163e+06,4.584405e+06,9.966988e+06,1.017294e+07,1.101388e+07
House Number,342008.0,7.663119e+03,5.439437e+03,1.000000e+00,2.504000e+03,7.916000e+03,1.142500e+04,5.117500e+04
Neighbourhood ID,384213.0,4.270034e+03,1.748173e+03,1.010000e+03,2.660000e+03,4.540000e+03,5.576000e+03,8.888000e+03
Assessed Value,384217.0,3.084198e+05,2.837778e+05,0.000000e+00,1.700000e+05,3.055000e+05,3.975000e+05,4.391150e+07
Latitude,384217.0,5.351842e+01,6.840256e-02,5.333859e+01,5.345862e+01,5.351981e+01,5.357419e+01,5.371515e+01
Longitude,384217.0,-1.135125e+02,8.282889e-02,-1.137129e+02,-1.135691e+02,-1.135105e+02,-1.134477e+02,-1.132784e+02


#### Fliter Outliers

We are looking for houses. There's a bunch of data here for houses that are both insanely cheap - near 0, or insanely expensive. We aren't big money investors, so I think the properties over $1mill are not relevant to our examination. 

We don't want that stuff because, for our purposes, it isn't usefull, so we'll remove it - leaving us with only homes that cost a "normal" amount. 

In [8]:
#Filter out houses that are out of the range that we care about.
#I used under $50k or over $1mill - this decision is largely arbitrary, based on what we are actually (pretending) to use this data for.


About 50,000 properties removed. Seems reasonable. We can look at the distribution and see what is there...

You could use our original graphing techniques (thinkplot), the bigGraph function above, the seaborn plotting, or a combination of all. 
<br><br><br>
#### Visualize Distributions

In [9]:
#Create some plots to examine the distribution of the data. 


We have taken a look at the statistics and the distribution of the data. Can we draw any conclusions from what we've seen here? 

### Part 2: Split Data

Split the data, to look for tendencies by groups. We have several values we can use, I will start with the most simple - garage. 

In [10]:
#Split the data into subsets. I used garage/not for my subsets. 


In [11]:
#Plot the subsets to look at their individual distributions. 


We can see that both of our slices of data, once separated, are much closer to a normal distribution. Analyzing these two separately is probably going to be easier than the double bump original distribution. 
I will caclulate a bunch of statistics for each group - including skew, so I can get a measure of how "stretched out" the distribution is. 

I will also look at each group and figure out what share of the homes are less than $200k, $350k, and $500k, for my own info - if I want to invest in properties that are "lowish" in value, how many are there?

### Part 3: Singular Statistics and CDF

In [12]:
#Example of printing out a bunch of statistics on one line


#### CDFs

We can use the CDF functions to figure out some statistics that we saw in the visualization of our data - what share of the data falls under a certain cut-off. 

In [13]:
#Use CDF to figure out how many are under each value cutoff. 


#### Initial Conclusions

What can we make of this? A few things, we aren't entirely sure if they will be important right now.

- Non garage houses are about $200k cheaper on average.

- Non garage houses are more tightly packed (smaller std), more "peaky" (kurtosis), and trail off to the right more (skew). This should kind of make sense logically, there's lots of smaller and cheaper nonm-garage homes, then there are a handful of luxury properties that scatter around the higher values. We can hypothesize that many/most non-garage houses are condos, but we need to check it. Houses over a certain, relatively low, value tend to always have garages so there's less weirdness to the distribution. 

- There is a comparatively massive cache of non-garage homes that are under $200k, and almost all are under $350k

#### Condo vs House - Proxy Data

Our dataset doesn't have a direct measure of if a piece of property is a condo or a house. Based on our expert knowledge, this factor probably helps in predicting value. What do we do????

We don't have a condo/house variable in the data, but we can use some other pieces of data as a proxy. This won't be 100% accurate, but it may be usefull enough to help inform our decision making. 

In [14]:
# Use suite values to assess house/condo split. My assumption is that if a home has a Suite number, it is a condo; if it doesn't, it is a house.
# If I calculate that percentage for each group, it could help indicate if my guess is true:

# Example of calculating how many of each set have no suite number. 
# print("Garage:", df_gar["Suite"].isnull().sum()/len(df_gar["Suite"]))
# print("Non:", df_no["Suite"].isnull().sum()/len(df_no["Suite"]))

In [15]:
# Graph the suite/nonsuite groups against eachother. 

I think my hypothesis is supported - non garage houses mostly do have a suite number, garage ones mainly don't. This isn't a 100% definitive result, but it is pretty solid. 

What can we take from this? Condos are cheaper, houses are more expensive. Garage is an OK-ish proxy for the house/condo split, not at all perfect - I also suspect it is probably a stronger relationship as prices rise, I doubt many expensive houses don't have garages. 

As well, perhaps as the city grows and as it becomes more dense and urban, perhaps the value of condos will rise and become closer to homes. We see a pattern of central condos becomming more expensive in most larger cities, a hypothesis that a similar thing will happen here is pretty reasonable. 


NOTE: At this point, we could also/alternatively split the data into suite/non-suite segments, or a 4 way split with both suite and garage. I will continue with garage/non for now. In real life, there's a pretty high probability that we'd come back and also split the data by suite. 

### Part 4: Investigate location. 

To investigate this more, we need to take a look at where these properties are located.... We can see what data we have - Ward, neighborhood, and lat/lon all indicate position. I need to go look at a map to see what is useful. Wards are large, and there are 12 of them. Neigborhoods are small, and there are a lot. We can start with wards and do a crude split for initial investigation. 

Note: We'd probably want to investigate things like square footage at this point in real life. We'll look at that comming up soon - looking at how two variables are related (correlated) to each other. 

In [16]:
#How many properties are in each ward? 


On a map, Ward 6 is downtown, it has the most non garage properties, which makes sense. 9 is Riverbend, there's lots of condos in Terwilligar and area. 4 is the far north east, 1 is the north-west, 11 is near south-east, 10 is Calgary Trail and west, 12 is far south east, and 8 is Strathcona and east to the city limit. 

First try, we can isolate downtown from the surroundings.

Note: I just googled a ward map. The ridings are changing for this current election - this data uses the old ridings, so I made sure I had an old map

In [17]:
#Create new dataframe for Ward 6, and everything else from the non-garage set of homes.

In [18]:
#Plot the new groupings. 


Bingo, bango, we may have found what we are looking for! Our downtown condos have a peak that is lower than the surrounding areas (probably largely because they are older). With our above hypothesis, we can draw some conclusions. The distribution is a little bimodal, that might require some more in depth investigation. I suspect that the lower bump are properties on the beltline of central Edmonton, but we need to investigate to be sure. 

If central areas become more valuable as cities grow and densify, then looking at investing in downtown, non-garage properties may make some sense. They appear to be undervalued here compared to other subsets of property. 

This also gives us some evidence that the prices of our non-garage condos are not "capped". They follow a normalish distribution, so we shouldn't expect that their price hits a ceiling if we renovate or as areas gentrify. 

In [19]:
#What percent of homes are 'cheap' in our different groups?
#Create CDFs and caclulate the totals below some cutoff - I used $150k for mine. We could do a few. 


In [20]:
#What percent of homes are 'cheap' in our different groups?


### Part 5: Neighborhoods

Investigate the location data a little further - examine what neighborhood these homes fall into. 

In [21]:
#How many homes are in each hood?


In [22]:
#What is the average value of each set of hood's homes?


In [23]:
#Combo (sort of) view of the above tables. 


### Part 6: Deep Dive on one Segment

Look at one potential area - Glenora. There are a non trivial number of homes, and it appears that there are several homes that are "underpriced" compared to what we may typically expect. 

If there are cheap non-garage homes in a very nice and generally expensive hood, we might think that if we were to improve these homes and/or wait for the city to grow and densify, that the value of these homes may "catch up" to other homes in the area. Many people want to live in Glenora, but the large houses are appreciating away from the affordability of more and more people. This isn't a fact, it is a pretty reasonable hypothesis. If you have ever heard the saying "buy the worst house on the block", it follows the same logic - location matters over anythig else. 

In [24]:
#Extract the glenora homes into their own dataframe


In [ ]:
#Create dataframe with all the garaged homes in Glenora, so we can compare. 


In [ ]:
#Plot Glenora's homes - garage vs none.


In [ ]:
#Take the satistics:


### Final (as of now) Conclusions

So, where are we? We have a hood where there are a non-trivial number of garge free homes, where the value of those homes is drastically lower than the overall average price in that hood. We also have a handfull of examples of expensive condos in that hood. We know this area is centrally located.

From external knowledge, glenora is a desireable and wealthy hood overall. 

If we are going to look into actual investments, the limited dataset we have here supports us researching Glenora, specifically homes without garages in Glenora as a potential target. We can't be sure that this makes sense without going out anf getting more domain knowledge though - our data didn't have everything - square footage, building age, etc... that we'd need to put cash on the line. 